In [1]:
using BenchmarkTools

# Curve 2

## Basic Definition

In [2]:
using Interpolations

In [3]:
struct Curve{Tx <: AbstractArray, Ty <: AbstractArray{<: Number}, 
        Titp <: Interpolations.AbstractInterpolation}
    x:: Tx
    y:: Ty
    etp:: Titp
    logx:: Bool
    logy:: Bool
end 

In [4]:
Curve(x, y; method=Gridded(Linear()), logx=false, logy=false, extrapolation=Flat()) =
    Curve(x, y, extrapolate(interpolate(logx ? (log.(x),) : (x,), logy ?  log.(y) : y, method), 
        extrapolation), logx, logy)

Curve

In [5]:
getitpm(c1:: Curve) = c1.etp.itp.it

getitpm (generic function with 1 method)

In [6]:
getetpm(c1:: Curve) = c1.etp.et

getetpm (generic function with 1 method)

In [7]:
Base.Broadcast.broadcastable(q:: Curve) = Ref(q) # treat it as a scalar in broadcasting

### Tests

In [8]:
c1 = Curve([3, 9, 18, 30, 91], [1.01, 1.204, 1.54, 1.81, 2.12])

Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([3, 9, 18, 30, 91], [1.01, 1.204, 1.54, 1.81, 2.12], 5-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
 1.01 
 1.204
 1.54 
 1.81 
 2.12 , false, false)

In [9]:
@btime c1 = Curve([3, 9, 18, 30, 91], [1.01, 1.204, 1.54, 1.81, 2.12])

  702.041 ns (6 allocations: 368 bytes)


Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([3, 9, 18, 30, 91], [1.01, 1.204, 1.54, 1.81, 2.12], 5-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
 1.01 
 1.204
 1.54 
 1.81 
 2.12 , false, false)

In [10]:
@allocated c1 = Curve([3, 9, 18, 30, 91], [1.01, 1.204, 1.54, 1.81, 2.12])

368

In [11]:
clogx = Curve([3, 9, 18, 30, 91], [1.01, 1.204, 1.54, 1.81, 2.12], logx=true, logy=true)

Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Float64,1}}},Gridded{Linear},Flat{Nothing}}}([3, 9, 18, 30, 91], [1.01, 1.204, 1.54, 1.81, 2.12], 5-element extrapolate(interpolate((::Array{Float64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
 0.009950330853168092
 0.18564934688662926 
 0.43178241642553783 
 0.5933268452777344  
 0.7514160886839212  , true, true)

In [12]:
@allocated clogx = Curve([3, 9, 18, 30, 91], [1.01, 1.204, 1.54, 1.81, 2.12], logx=true, logy=true)

624

In [13]:
@btime clogx = Curve([3, 9, 18, 30, 91], [1.01, 1.204, 1.54, 1.81, 2.12], logx=true, logy=true)

  1.106 μs (8 allocations: 624 bytes)


Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Float64,1}}},Gridded{Linear},Flat{Nothing}}}([3, 9, 18, 30, 91], [1.01, 1.204, 1.54, 1.81, 2.12], 5-element extrapolate(interpolate((::Array{Float64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
 0.009950330853168092
 0.18564934688662926 
 0.43178241642553783 
 0.5933268452777344  
 0.7514160886839212  , true, true)

## Interpolation

In [14]:
import Interpolations: interpolate

In [15]:
function interpolate(xval:: Real, c1:: Curve)
    c = c1.etp(c1.logx ? log(xval) : xval)
    c1.logy ? exp(c) : c
end

interpolate (generic function with 11 methods)

In [16]:
interpolate(xval:: AbstractArray{T} where T, c1:: Curve) = 
    Curve(xval, interpolate.(xval, c1), method=getitpm(c1), logx=c1.logx, logy=c1.logy, 
        extrapolation=getetpm(c1))

interpolate (generic function with 12 methods)

In [17]:
interpolate(c0:: Curve, c1:: Curve) = interpolate(c0.x, c1)

interpolate (generic function with 13 methods)

### Tests

In [18]:
@btime interpolate(5.5, $c1)

  71.117 ns (0 allocations: 0 bytes)


1.0908333333333333

In [19]:
@btime interpolate([3, 9, 14, 31, 33], $c1)

  887.396 ns (6 allocations: 368 bytes)


Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([3, 9, 14, 31, 33], [1.01, 1.204, 1.3906666666666667, 1.8150819672131149, 1.825245901639344], 5-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
 1.01              
 1.204             
 1.3906666666666667
 1.8150819672131149
 1.825245901639344 , false, false)

In [20]:
@btime interpolate([3, 9, 14, 31, 33], $clogx)

  1.788 μs (8 allocations: 624 bytes)


Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Float64,1}}},Gridded{Linear},Flat{Nothing}}}([3, 9, 14, 31, 33], [1.01, 1.204, 1.40852341161862, 1.8184750835405903, 1.834744648303743], 5-element extrapolate(interpolate((::Array{Float64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
 0.009950330853168092
 0.18564934688662926 
 0.3425419298636137  
 0.5979982836961524  
 0.6069053155856639  , true, true)

## Define Operations with Scalars

In [21]:
import Base: +, -, *, /, ^, exp, log, length

In [22]:
length(c1:: Curve):: Int = length(c1.y)

length (generic function with 94 methods)

In [23]:
operations = (:+, :-, :*, :/, :^)
for op in operations
    @eval $op(c1:: Curve, a:: Number) = Curve(c1.x, $op.(c1.y, a), 
        method=getitpm(c1), logx=c1.logx, logy=c1.logy, extrapolation=getetpm(c1))
    @eval $op(a:: Number, c1:: Curve) = Curve(c1.x, $op.(a, c1.y), 
        method=getitpm(c1), logx=c1.logx, logy=c1.logy, extrapolation=getetpm(c1))
end

In [24]:
operations = (:exp, :log)
for op in operations
    @eval $op(c1:: Curve) = Curve(c1.x, $op.(c1.y), method=getitpm(c1), 
        logx=c1.logx, logy=c1.logy, extrapolation=getetpm(c1))
end

### Tests

In [25]:
length(c1)

5

In [26]:
@code_warntype c1 + 2

Variables
  #self#::Core.Compiler.Const(+, false)
  c1::Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}
  a::Int64

Body::Union{Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Float64,1}}},Gridded{Linear},Flat{Nothing}}}, Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}}
1 ─ %1  = Main.getitpm(c1)::Core.Compiler.Const(Gridded(Linear()), false)
│   %2  = Main.getetpm(c1)::Core.Compiler.Const(Flat(), false)
│   %3  = (:method, :logx, :logy, :extrapolation)::Core.Compiler.Const((:method, :logx, :logy, :extrapolation), false)
│   %4  = Core.apply_type(Core.NamedTuple, %3)::Core.C

In [27]:
@btime c1 + 2

  620.663 ns (5 allocations: 240 bytes)


Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([3, 9, 18, 30, 91], [3.01, 3.2039999999999997, 3.54, 3.81, 4.12], 5-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
 3.01              
 3.2039999999999997
 3.54              
 3.81              
 4.12              , false, false)

In [28]:
1/c1

Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([3, 9, 18, 30, 91], [0.9900990099009901, 0.8305647840531561, 0.6493506493506493, 0.5524861878453039, 0.4716981132075471], 5-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
 0.9900990099009901
 0.8305647840531561
 0.6493506493506493
 0.5524861878453039
 0.4716981132075471, false, false)

In [29]:
@btime exp(c1)

  786.347 ns (5 allocations: 240 bytes)


Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([3, 9, 18, 30, 91], [2.7456010150169163, 3.3334239868128988, 4.664590270988126, 6.11044743223061, 8.331137487687693], 5-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
 2.7456010150169163
 3.3334239868128988
 4.664590270988126 
 6.11044743223061  
 8.331137487687693 , false, false)

## Concatination

In [111]:
@inline function uniquexy(x:: AbstractArray, y:: AbstractArray)
    @inbounds begin
        idx = unique(z -> x[z], 1:length(x))
        x[idx], y[idx] 
    end
end

uniquexy (generic function with 1 method)

In [85]:
@inline function mergexy(x1:: AbstractArray, y1:: AbstractArray, 
        x2:: AbstractArray, y2:: AbstractArray)
    @inbounds begin
        x_all = vcat(x1, x2)
        y_all = vcat(y1, y2)
        perm = sortperm(x_all)
        x_all[perm], y_all[perm]
    end
end

mergexy (generic function with 2 methods)

In [71]:
"""
Removes duplicate x values from the curve.
Note that it is not checked if the corresponding y values are identical, 
just an arbitrary one is kept.
"""
function drop_duplicates(c1:: Curve)
    x, y = uniquexy(c1.x, c1.y)
    Curve(x, y, method=getitpm(c1), logx=c1.logx, logy=c1.logy, extrapolation=getetpm(c1))
end

drop_duplicates (generic function with 1 method)

In [77]:
"""
Merges 2 curves.
Element type is inferred by promotion, interpolation type is taken from 1st curve argument.
"""
function concat(c1:: Curve, c2:: Curve; drop_dup=true)
    x_all, y_all = mergexy(c1.x, c1.y, c2.x, c2.y)
    if drop_dup
        x_all, y_all = uniquexy(x_all, y_all)
    end
    Curve(x_all, y_all, method=getitpm(c1), logx=c1.logx, logy=c1.logy, 
        extrapolation=getetpm(c1))
end

concat

### Tests

In [115]:
c1d = Curve([1, 3, 3, 7, 9], [2, 4, 4, 8, 10])

Curve{Array{Int64,1},Array{Int64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Int64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([1, 3, 3, 7, 9], [2, 4, 4, 8, 10], 5-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Int64,1}, Gridded(Linear())), Flat()) with element type Float64:
 Ratios.SimpleRatio{Int64}(8, 4)   
 Ratios.SimpleRatio{Int64}(16, 4)  
 Ratios.SimpleRatio{Int64}(16, 4)  
 Ratios.SimpleRatio{Int64}(128, 16)
 Ratios.SimpleRatio{Int64}(40, 4)  , false, false)

In [116]:
drop_duplicates(c1d)

Curve{Array{Int64,1},Array{Int64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Int64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([1, 3, 7, 9], [2, 4, 8, 10], 4-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Int64,1}, Gridded(Linear())), Flat()) with element type Float64:
 Ratios.SimpleRatio{Int64}(8, 4)   
 Ratios.SimpleRatio{Int64}(16, 4)  
 Ratios.SimpleRatio{Int64}(128, 16)
 Ratios.SimpleRatio{Int64}(40, 4)  , false, false)

In [117]:
@btime drop_duplicates(c1d)

  1.583 μs (14 allocations: 976 bytes)


Curve{Array{Int64,1},Array{Int64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Int64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([1, 3, 7, 9], [2, 4, 8, 10], 4-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Int64,1}, Gridded(Linear())), Flat()) with element type Float64:
 Ratios.SimpleRatio{Int64}(8, 4)   
 Ratios.SimpleRatio{Int64}(16, 4)  
 Ratios.SimpleRatio{Int64}(128, 16)
 Ratios.SimpleRatio{Int64}(40, 4)  , false, false)

In [114]:
@code_warntype drop_duplicates(c1d)

Variables
  #self#::Core.Compiler.Const(drop_duplicates, false)
  c1::Curve{Array{Int64,1},Array{Int64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Int64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}
  x::Array{Int64,1}
  @_4::Int64
  y::Array{Int64,1}

Body::Curve{Array{Int64,1},Array{Int64,1},_A} where _A<:AbstractInterpolation
1 ─ %1  = Base.getproperty(c1, :x)::Array{Int64,1}
│   %2  = Base.getproperty(c1, :y)::Array{Int64,1}
│   %3  = Main.uniquexy(%1, %2)::Tuple{Array{Int64,1},Array{Int64,1}}
│   %4  = Base.indexed_iterate(%3, 1)::Core.Compiler.PartialStruct(Tuple{Array{Int64,1},Int64}, Any[Array{Int64,1}, Core.Compiler.Const(2, false)])
│         (x = Core.getfield(%4, 1))
│         (@_4 = Core.getfield(%4, 2))
│   %7  = Base.indexed_iterate(%3, 2, @_4::Core.Compiler.Const(2, false))::Core.Compiler.PartialStruct(Tuple{Array{Int64,1},Int64}, Any[Array{Int64,1}, Core.Compiler.Const(3, false)])
│         (y = Core.

In [86]:
c12 = concat(c1,c1d)

Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([1, 3, 7, 9, 18, 30, 91], [2.0, 1.01, 8.0, 1.204, 1.54, 1.81, 2.12], 7-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
 2.0  
 1.01 
 8.0  
 1.204
 1.54 
 1.81 
 2.12 , false, false)

In [82]:
c12.x, c12.y

([1, 3, 7, 9, 18, 30, 91], [2.0, 1.01, 8.0, 1.204, 1.54, 1.81, 2.12])

In [87]:
@btime concat($c1,$c1d)

  2.804 μs (22 allocations: 1.92 KiB)


Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([1, 3, 7, 9, 18, 30, 91], [2.0, 1.01, 8.0, 1.204, 1.54, 1.81, 2.12], 7-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
 2.0  
 1.01 
 8.0  
 1.204
 1.54 
 1.81 
 2.12 , false, false)

In [84]:
@code_warntype concat(c1,c1d)

Variables
  #self#::Core.Compiler.Const(concat, false)
  c1::Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}
  c2::Curve{Array{Int64,1},Array{Int64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Int64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}

Body::Union{Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Float64,1}}},Gridded{Linear},Flat{Nothing}}}, Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}}
1 ─ %1 = Main.:(var"#concat#29")(true, #self#, c1, c2)::Union{Curve{Array{Int64,1},Array{Float64,1},Interpolatio

## Operations with Multiple Curves

In [127]:
operations = (:+, :-, :*, :/, :^)
for op in operations
    @eval function $op(c1:: Curve, c2:: Curve)
        if c1.x == c2.x # fast mode if no interpolation is required
            x = c1.x
            y = $op.(c1.y, c2.y)
        else
            y_c1_grid = $op.(c1.y, interpolate.(c1.x, c2))
            y_c2_grid = $op.(interpolate.(c2.x, c1), c2.y)
            x, y = mergexy(c1.x, y_c1_grid, c2.x, y_c2_grid)
            x, y = uniquexy(x, y)
        end
        Curve(x, y, logx=c1.logx, logy=c1.logy, method=getitpm(c1), extrapolation=getetpm(c1))
    end
end


### Tests

In [128]:
sumc12 = c1+c1d

Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([1, 3, 7, 9, 18, 30, 91], [3.01, 5.01, 9.139333333333333, 11.204, 11.54, 11.81, 12.120000000000001], 7-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
  3.01             
  5.01             
  9.139333333333333
 11.204            
 11.54             
 11.81             
 12.120000000000001, false, false)

In [129]:
c1^ c1d

Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([1, 3, 7, 9, 18, 30, 91], [1.0201, 1.04060401, 2.8392682718042925, 6.4012511846901345, 75.02520803928272, 377.38596846955716, 1833.828041259883], 7-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
    1.0201            
    1.04060401        
    2.8392682718042925
    6.4012511846901345
   75.02520803928272  
  377.38596846955716  
 1833.828041259883    , false, false)

In [130]:
@btime log(c1/c1d)

  4.674 μs (30 allocations: 2.42 KiB)


Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([1, 3, 7, 9, 18, 30, 91], [-0.6831968497067772, -1.3763440302667225, -1.9489982456545465, -2.1169357461074165, -1.870802676568508, -1.7092582477163114, -1.5511690043101245], 7-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
 -0.6831968497067772
 -1.3763440302667225
 -1.9489982456545465
 -2.1169357461074165
 -1.870802676568508 
 -1.7092582477163114
 -1.5511690043101245, false, false)

In [131]:
sumc12.x, sumc12.y

([1, 3, 7, 9, 18, 30, 91], [3.01, 5.01, 9.139333333333333, 11.204, 11.54, 11.81, 12.120000000000001])

In [132]:
@btime $c1+$c1d

  3.611 μs (25 allocations: 2.17 KiB)


Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([1, 3, 7, 9, 18, 30, 91], [3.01, 5.01, 9.139333333333333, 11.204, 11.54, 11.81, 12.120000000000001], 7-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
  3.01             
  5.01             
  9.139333333333333
 11.204            
 11.54             
 11.81             
 12.120000000000001, false, false)

In [133]:
@btime $c1+$c1

  599.438 ns (5 allocations: 240 bytes)


Curve{Array{Int64,1},Array{Float64,1},Interpolations.Extrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Gridded{Linear},Flat{Nothing}}}([3, 9, 18, 30, 91], [2.02, 2.408, 3.08, 3.62, 4.24], 5-element extrapolate(interpolate((::Array{Int64,1},), ::Array{Float64,1}, Gridded(Linear())), Flat()) with element type Float64:
 2.02 
 2.408
 3.08 
 3.62 
 4.24 , false, false)